In [1]:
from GlucoCheck.glucoCheck import glucoCheckOps
glucoCheckObject = glucoCheckOps()

Using TensorFlow backend.


In [2]:
glucoCheckObject.all_data

,subjectId,Display Time,GlucoseValue
0,GVP01,6/21/16 00:00,197.0
1,GVP01,6/21/16 00:15,208.0
2,GVP01,6/21/16 00:30,226.0
3,GVP01,6/21/16 00:45,241.0
4,GVP01,6/21/16 01:00,250.0
...,...,...,...
350017,OD552,6/7/25 16:49,238.0
350018,OD552,6/7/25 16:54,233.0
350019,OD552,6/7/25 16:59,229.0
350020,OD552,6/7/25 17:04,224.0


In [3]:
glucoCheckObject.metadata

,ID,status
0,ID01,diabetic
1,ID02,diabetic
2,ID03,diabetic
3,ID11,diabetic
4,ID12,diabetic
...,...,...
84,OD552,NAN
85,OD567,NAN
86,OD584,NAN
87,OD596,NAN


In [6]:
data = glucoCheckObject.hallData

In [5]:
glucoCheckObject.dataDescribe(glucoCheckObject.hallData)

,Subject ID,Start,End,# of readings,# of days,Timestamp days,# of missing values,% of missing values,Avg gap size,Max gv,Min gv,Mean gv
0,2133-041,2017-07-03 18:32:00,2017-07-11 20:41:00,1858,8.08,8 days 02:04:00,490,21.04,28.82,204,51,108.56
1,2133-040,2017-06-28 13:34:00,2017-07-05 00:19:00,1792,6.42,6 days 10:40:00,100,5.39,2.94,171,59,92.59
2,2133-039,2017-06-05 12:23:00,2017-06-14 13:57:00,2013,9.04,9 days 01:29:00,733,28.09,5.51,204,50,103.92
3,2133-037,2017-06-05 14:11:00,2017-06-11 21:50:00,1785,6.29,6 days 07:34:00,57,3.13,2.71,153,48,92.97
4,2133-036,2017-06-01 15:26:00,2017-06-10 06:31:00,1954,8.62,8 days 15:00:00,651,26.21,5.47,214,58,107.53
5,2133-035,2017-06-01 17:17:00,2017-06-08 23:06:00,1830,7.21,7 days 05:44:00,327,15.68,4.81,190,47,101.77
6,2133-033,2017-05-19 12:58:00,2017-05-26 20:02:00,1892,7.25,7 days 06:59:00,237,11.29,8.78,161,49,92.98
7,2133-032,2017-05-19 13:31:00,2017-05-25 18:05:00,1781,6.17,6 days 04:29:00,8,0.45,1.60,147,69,99.48
8,2133-030,2017-05-03 14:13:00,2017-05-10 08:57:00,1818,6.75,6 days 18:39:00,144,7.38,18.00,147,41,93.20
9,2133-028,2017-05-10 00:03:00,2017-05-17 02:08:00,1850,7.08,7 days 02:00:00,195,9.56,65.00,111,45,74.79


In [7]:
QC_table = glucoCheckObject.dataDescribe(data)
QC_table

,Subject ID,Start,End,# of readings,# of days,Timestamp days,# of missing values,% of missing values,Avg gap size,Max gv,Min gv,Mean gv
0,2133-041,2017-07-03 18:32:00,2017-07-11 20:41:00,1858,8.08,8 days 02:04:00,490,21.04,28.82,204,51,108.56
1,2133-040,2017-06-28 13:34:00,2017-07-05 00:19:00,1792,6.42,6 days 10:40:00,100,5.39,2.94,171,59,92.59
2,2133-039,2017-06-05 12:23:00,2017-06-14 13:57:00,2013,9.04,9 days 01:29:00,733,28.09,5.51,204,50,103.92
3,2133-037,2017-06-05 14:11:00,2017-06-11 21:50:00,1785,6.29,6 days 07:34:00,57,3.13,2.71,153,48,92.97
4,2133-036,2017-06-01 15:26:00,2017-06-10 06:31:00,1954,8.62,8 days 15:00:00,651,26.21,5.47,214,58,107.53
5,2133-035,2017-06-01 17:17:00,2017-06-08 23:06:00,1830,7.21,7 days 05:44:00,327,15.68,4.81,190,47,101.77
6,2133-033,2017-05-19 12:58:00,2017-05-26 20:02:00,1892,7.25,7 days 06:59:00,237,11.29,8.78,161,49,92.98
7,2133-032,2017-05-19 13:31:00,2017-05-25 18:05:00,1781,6.17,6 days 04:29:00,8,0.45,1.60,147,69,99.48
8,2133-030,2017-05-03 14:13:00,2017-05-10 08:57:00,1818,6.75,6 days 18:39:00,144,7.38,18.00,147,41,93.20
9,2133-028,2017-05-10 00:03:00,2017-05-17 02:08:00,1850,7.08,7 days 02:00:00,195,9.56,65.00,111,45,74.79
